# Tutorial - examples of all metrics

This notebook shows the functionality of the different metrics included in the library. 

For this purpose, we use a pre-trained ResNet-18 model and X dataset.

In [1]:
# Mount Google Drive. #remove
# from google.colab import drive 
# drive.mount('/content/drive', force_remount=True)

# Install packages. #remove
!pip install opencv-python torch==1.8.0+cu111 torchvision==0.9.0+cu111 captum -f https://download.pytorch.org/whl/torch_stable.html

# Imports general.
import sys
import warnings
import gc
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import captum
from torchvision import transforms
from tqdm import tqdm
from captum.attr import *
import random

# Import package.
# sys.path.append('/content/drive/MyDrive/Projects/xai_quantification_toolbox')
sys.path.append("../")
import quantus

# Notebook settings.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
warnings.filterwarnings("ignore", category=UserWarning)
%load_ext autoreload
%autoreload 2

# Collect garbage.
gc.collect()
torch.cuda.empty_cache()

Looking in links: https://download.pytorch.org/whl/torch_stable.html


## Preliminaries

### Load datasets

We will then load a batch of input, output pairs that we generate explanations for, then to evaluate.

In [2]:
# Load test data and make loaders.
data_path = "../../data/imagenet/imagenet/bboxes_images/"
test_set = torchvision.datasets.ImageFolder(root=data_path,
                                            transform=transforms.Compose([transforms.Resize(256),
                                                                          transforms.CenterCrop((224, 224)),
                                                                          transforms.ToTensor(),
                                                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]))
test_loader = torch.utils.data.DataLoader(test_set, shuffle=True, batch_size=24)

# Load a batch of inputs and outputs to use for evaluation.
x_batch, y_batch = iter(test_loader).next()
x_batch, y_batch = x_batch.to(device), y_batch.to(device)

### Load a torch model

In this example we load a pre-trained ResNet-18 model but it goes without saying that any model works.

In [3]:
def evaluate_model(model, data, device):
    """Evaluate accuracy of torch model."""
    model.eval()
    logits = torch.Tensor().to(device)
    targets = torch.LongTensor().to(device)

    with torch.no_grad():
        for images, labels in data:
            images, labels = images.to(device), labels.to(device)
            logits = torch.cat([logits, model(images)])
            targets = torch.cat([targets, labels])
            
    return np.mean(np.argmax(logits.cpu().numpy(), axis=1) == targets.cpu().numpy())
    
# Load pre-trained ResNet18 model.
model = torchvision.models.resnet18(pretrained=True)
model = model.to(device)

# Evaluate model performance.
#print(f"\nModel test accuracy: {(100 * evaluate_model(model.to(device), test_loader, device)):.2f}%")

### Generate attribution-based explanations

There exist multiple ways to generate explanations for neural network models e.g., using `captum` or `innvestigate` libraries. In this example, we rely on the `quantus.explain` functionality (a simple wrapper around `captum`) however use whatever approach or library you'd like to create your explanations.

**Requirements.**

* **Data type.** Similar to the x-y pairs, the attributions should also be of type `np.ndarray`
* **Shape.** Sharing all the same dimensions as the input (expect for nr_channels which for explanations is equal to 1). For example, if x_batch is of size (128, 3, 224, 224) then the attributions should be of size (128, 1, 224, 224).

In [4]:
# Generate Integrated Gradients attributions of the first batch of the test set.
a_batch = quantus.explain(model, x_batch, y_batch, explanation_func="Saliency")

../quantus/helpers/explanation_func.py:30: Warning: Using quantus 'explain' function as an explainer without specifying 'method' (str)in kwargs will produce a vanilla 'Gradient' explanation.

  f"in kwargs will produce a vanilla 'Gradient' explanation.\n", category=Warning)


In [5]:
# Save x_batch and y_batch as numpy arrays that will be used to call metric instances.
x_batch, y_batch = x_batch.cpu().numpy(), y_batch.cpu().numpy()

# Quick assert.
assert [isinstance(obj, np.ndarray) for obj in [x_batch, y_batch, a_batch]]

## Evaluation using `quantus`

As a preliminary step to evaluating the explanations using all the available metrics in the library, we will first view what categories and metrics we have available.

In [21]:
# Let's view what metrics categories are available.
quantus.available_categories()

['Faithfulness',
 'Robustness',
 'Localisation',
 'Complexity',
 'Randomisation',
 'Axiomatic']

In [22]:
# In each category, let's view the available metrics.
metrics = quantus.available_metrics()
metrics

{'Faithfulness': ['Faithfulness Correlation',
  'Faithfulness Estimate',
  'Pixel-Flipping',
  'Region Segmentation',
  'Monotonicity Arya',
  'Monotonicity Nguyen',
  'Infidelity',
  'Selectivity',
  'SensitivityN',
  'IROF'],
 'Robustness': ['Continuity Test',
  'Input Independence Rate',
  'Local Lipschitz Estimate',
  'Max-Sensitivity',
  'Avg-Sensitivity'],
 'Localisation': ['Pointing Game',
  'TKI',
  'Relevance Mass Accuracy',
  'Relevance Mass Ranking',
  'Attribution Localization '],
 'Complexity': ['Sparseness Test', 'Complexity Test', 'Effective Complexity'],
 'Randomisation': ['Model Parameter Randomisation Test', 'Random Logit Test'],
 'Axiomatic': ['Completeness Test',
  'Symmetry',
  'InputInvariance',
  'NonSensitivity',
  'Dummy']}

#### Faithfulness metrics 

Score explanations using the following faithfulness metrics:

* Faithfulness Correlation (Bhatt et al., 2020)
* Faithfulness Estimate (Alvarez-Melis et al., 2018, 2019)
* Pixel-Flipping (Bach et al., 2015)
* Region Segmentation (Samek et al., 2015)
* Monotonicity-Arya (Arya et al., 2019)
* Monotonicity-Nguyen (Nguyen et al., 2019)
* Infidelity (Yeh et al., 2010)
* Selectivity (Montavan et al., 2018)
* SensitivityN (Ancona et al., 2018)
* IROF (Rieger et al., 2020)

In [9]:
# Return faithfulness correlation scores in an one-liner - by calling the metric instance.
quantus.FaithfulnessCorrelation(**{
    "nr_runs": 100,  
    "subset_size": 224,  
    "perturb_baseline": "black",
    "perturb_func": quantus.baseline_replacement_by_indices,
    "similarity_func": quantus.correlation_pearson,    
})(model=model, 
   x_batch=x_batch, 
   y_batch=y_batch,
   a_batch=a_batch,
   **{"device": device})

[0.003654576425917891]

In [10]:
# Return faithfulness estimate scores in an one-liner - by calling the metric instance.
quantus.FaithfulnessEstimate(**{
    "perturb_func": quantus.baseline_replacement_by_indices,
    "similarity_func": quantus.correlation_pearson,
    "img_size": 224,  
    "features_in_step": 224,  
    "perturb_baseline": "black",  
    "pixels_in_step": 28,
})(model=model, 
   x_batch=x_batch, 
   y_batch=y_batch,
   a_batch=a_batch,
   **{"device": device})

[0.5334162290691334,
 0.4577095276998041,
 0.04839912029139247,
 0.6541374660573284,
 -0.21870232522596827,
 -0.0953440312178733,
 0.48638221187365677,
 0.37048781898147654,
 0.36713468105665725,
 0.16607231006097126,
 -0.0743457234135028,
 -0.5516383653651101,
 0.30093702858647997,
 0.19297733278908288,
 -0.15239167839096307,
 0.34747207953033415,
 -0.4365920722337685,
 0.4887104426610205,
 0.6248802203767071,
 0.3971723928932323,
 0.2420483251349218,
 -0.133391571018306,
 -0.21966653906404737,
 -0.4600634159278445]

In [11]:
# Return monotonicity scores in an one-liner - by calling the metric instance.
quantus.MonotonicityArya(**{
    "features_in_step": 224,
    "perturb_baseline": "black",
    "perturb_func": quantus.baseline_replacement_by_indices,
    "similarity_func": quantus.correlation_spearman,
})(model=model, 
   x_batch=x_batch, 
   y_batch=y_batch,
   a_batch=a_batch,
   **{"device": device})

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [12]:
# Return monotonicity scores in an one-liner - by calling the metric instance.
quantus.MonotonicityNguyen(**{
    "n_samples": 100,
    "perturb_baseline": "uniform",
    "features_in_step": 224,
    "perturb_func": quantus.baseline_replacement_by_indices,
    "similarity_func": quantus.correlation_spearman,
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=a_batch,
   **{"device": device})

KeyboardInterrupt: 

In [14]:
# Return pixel-flipping scores in an one-liner - by calling the metric instance.
scores = quantus.PixelFlipping(**{
    "features_in_step": 28,
    "perturb_baseline": "black",
    "perturb_func": quantus.baseline_replacement_by_indices,
})(model=model,
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=a_batch,
   **{"device": device})

# Plot example!
quantus.plot_pixel_flipping_experiment(y_batch=y_batch, scores=scores)

KeyboardInterrupt: 

In [ ]:
# Return region perturbation scores in an one-liner - by calling the metric instance.
results = {method: quantus.RegionPerturbation(**{
    "patch_size": 28,
    "regions_evaluation": 100,
    "img_size": 224,
    "random_order": False,
    "perturb_func": quantus.baseline_replacement_by_patch,
    "perturb_baseline": "uniform",  
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"explain_func": quantus.explain, "method": method, "device": device}) for method in ["Saliency", "IntegratedGradients"]}

# Plot example!
quantus.plot_region_perturbation_experiment(results=results)

In [ ]:
# Return selectivity scores in an one-liner - by calling the metric instance.
results = {method: quantus.Selectivity(**{
    "patch_size": 14,
    "perturb_func": quantus.baseline_replacement_by_patch,
    "perturb_baseline": "black",  
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"explain_func": quantus.explain, "method": method, "device": device}) for method in ["Saliency", "IntegratedGradients"]}

# Plot example!
quantus.plot_selectivity_experiment(results=results)

In [ ]:
# Return sensitivity-n scores in an one-liner - by calling the metric instance.
scores = quantus.SensitivityN(**{
    "features_in_step": 28,
    "n_max_percentage": 0.8,
    "img_size": 224,
    "similarity_func": quantus.correlation_pearson,
    "perturb_func": quantus.baseline_replacement_by_indices,
    "perturb_baseline": "uniform",  
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=a_batch,
   **{"device": device})

# Plot example!
quantus.plot_sensitivity_n_experiment(results=scores)

# TODO. Validate the curves - are they looking as expected?

In [ ]:
# Return IFOF scores in an one-liner - by calling the metric instance.
results = {method: quantus.IROF(**{
    "segmentation_method": "slic",
    "perturb_baseline": "mean",
    "perturb_func": quantus.baseline_replacement_by_indices,
})(model=model,
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"explain_func": quantus.explain, "method": method, "device": device}) for method in ["Gradient", "Saliency", "IntegratedGradients", "GradientShap"]}

results

In [ ]:
# TODO. Bug fixes.
'''
# Return infidelity scores in an one-liner - by calling the metric instance.
quantus.Infidelity(**{
    "loss_func": quantus.mse,
    "img_size": 224,
    "perturb_patch_sizes": [8, 14, 28],
    "perturb_baseline": "black",  
    "perturb_func": quantus.baseline_replacement_ (_patch,
})(model=model, 
   x_batch=x_batch, 
   y_batch=y_batch,
   a_batch=a_batch,
   **{"device": device})
''';

#### Robustness metrics 

Score explanations using the following robustness metrics:

* Local Lipschitz Estimate (Alvarez-Melis et al., 2018; 2019)
* Max-Sensitivity (Yeh at el., 2019)
* Avg-Sensitivity (Yeh at el., 2019)
* Continuity (Montavan et al., 2017)
* Input Independence Rate (Yang et al., 2019)

In [ ]:
# Return local lipschitzEstimate estimate scores in an one-liner - by calling the metric instance.
quantus.LocalLipschitzEstimate(**{
    "nr_samples": 10,
    "perturb_std": 0.1,
    "norm_numerator": quantus.distance_euclidean,
    "norm_denominator": quantus.distance_euclidean,
    "explain_func": quantus.explain,
    "perturb_func": quantus.gaussian_noise,
    "similarity_func": quantus.lipschitz_constant,
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"method": "Saliency", "device": device})

In [ ]:
# Return max sensitivity scores in an one-liner - by calling the metric instance.
quantus.MaxSensitivity(**{
    "nr_samples": 10,
    "perturb_radius": 0.2,
    "norm_numerator": quantus.fro_norm,
    "norm_denominator": quantus.fro_norm,
    "explain_func": quantus.explain,
    "perturb_func": quantus.uniform_sampling,
    "similarity_func": quantus.difference,
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"method": "Saliency", "device": device})

In [ ]:
# Return avg sensitivity scores in an one-liner - by calling the metric instance.
quantus.AvgSensitivity(**{
    "nr_samples": 10,
    "perturb_radius": 0.2,
    "norm_numerator": quantus.fro_norm,
    "norm_denominator": quantus.fro_norm,
    "explain_func": quantus.explain,
    "perturb_func": quantus.uniform_sampling,
    "similarity_func": quantus.difference,
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"method": "Saliency", "device": device})

In [ ]:
# Return continuity scores in an one-liner - by calling the metric instance.
quantus.Continuity(**{
    "nr_patches": 4,
    "nr_steps": 10,
    "img_size": 224,
    "perturb_baseline": "black",
    "explain_func": quantus.explain,
    "similarity_func": quantus.correlation_spearman,
    "perturb_func": quantus.translation_x_direction,
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"method": "IntegratedGradients", "device": device})

#print(f"Continutity correlation score: {continuity_test.continutity_correlation_score:.4f}")

In [ ]:
# TODO. Remove from library - iteration 1.
'''
# Return input independence rate scores in an one-liner - by calling the metric instance.
quantus.InputIndependenceRate(**{
    "similarity_func": abs_difference,
    "perturb_func": optimization_scheme, # TODO.
    "perturb_std": 0.01,
    "threshold": 0.025, # 0.1
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=a_batch,
   **{"explanation_func": explain, "method": method, "device": device})
''';

### Randomization metrics

In [24]:
# Return model parameter randomization scores in an one-liner - by calling the metric instance.
methods = ["Saliency", "Gradient"]

results = {method: quantus.ModelParameterRandomization(**{
    "layer_order": "independent",
    "similarity_func": quantus.correlation_spearman,
    "explain_func": quantus.explain, 
    "normalize": True,
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"method": method, "device": device}) for method in methods}


quantus.plot_model_parameter_randomization_experiment(results=results, methods=methods)

In [29]:
# Return random logit scores in an one-liner - by calling the metric instance.
quantus.RandomLogit(**{
    "num_classes": 1000,
    "similarity_func": quantus.ssim,
    "explain_func": quantus.explain, 
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"method": "Saliency", "device": device})

torch.Size([64, 3, 7, 7])


array([0.75890981, 0.79094196, 0.76224042, 0.7633094 , 0.78604279,
       0.76436734, 0.76418971, 0.78765934, 0.77336132, 0.75951437,
       0.79109952, 0.78600739, 0.79397298, 0.80506919, 0.77071526,
       0.77258477, 0.76996429, 0.79115599, 0.76670788, 0.78373212,
       0.79335446, 0.76389674, 0.75930943, 0.76438664])

### Complexity metrics

Score explanations using the following complexity metrics:

* Sparseness (Alvarez-Melis et al., 2018; 2019)
* Complexity (Yeh at el., 2019)
* Effective complexity (Yeh at el., 2019)

In [ ]:
# Return sparseness scores in an one-liner - by calling the metric instance.
quantus.Sparseness(**{
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"explain_func": quantus.explain, "method": "IntegratedGradients", "device": device})

In [ ]:
# Return complexity scores in an one-liner - by calling the metric instance.
quantus.Complexity(**{
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"explain_func": quantus.explain, "method": "IntegratedGradients", "device": device})

In [ ]:
# Return effective complexity scores in an one-liner - by calling the metric instance.
quantus.EffectiveComplexity(**{
    "eps": 1e-5,
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"explain_func": quantus.explain, "method": "IntegratedGradients", "device": device})

### Axiomatic metrics

Score explanations using the following axiomatic metrics:

* Completeness (Montavon et al., 2018) - Additivity
* Non-sensitivity (Nguyen at el., 2020) - Dummy
* Symmetry
* Input invaraince

In [ ]:
# Return completeness scores in an one-liner - by calling the metric instance.
quantus.Completeness(**{
    "abs": False,
    #"output_func": torch.nn.Softmax(),
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"explain_func": quantus.explain, "method": "IntegratedGradients", "device": device})

In [ ]:
# Return non-sensitivity scores in an one-liner - by calling the metric instance.
quantus.NonSensitivity(**{
    "abs": True,
    "eps": 1e-5,
    "perturb_baseline": "black",
    "perturb_func": quantus.baseline_replacement_by_indices,
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   **{"explain_func": quantus.explain, "method": "IntegratedGradients", "device": device})

### Localisation metrics

* Pointing Game (Zhang et al., 2018)
* Attribution Localization (Kohlbrenner et al., 2020)
* TKI (Theiner et al., 2021)
* Relevance Rank Accuracy (Arras et al., 2021)
* Relevance Mass Accuracy (Arras et al., 2021)

In [3]:
# For the localisation metrics we need to load a dataset that has ground truth masks.
import os
import PIL
from xml.etree import ElementTree
import xmltodict
import cv2
import collections

def load_sample(path):
    """ Reads data as image and path. """
    return {"data": PIL.Image.open(path).convert("RGB"), "path": path}

def load_binary_mask(filename, datapath):
    """ Loads the binary mask for the given filepath in the correct format. """
    binary_mask = {}
    
    filename = os.path.splitext(filename)[0]
    
    # get label and file name
    label = filename.split("/")[-2]
    fname = filename.split("_")[-1]

    # parse annotations
    tree = ElementTree.parse(os.path.join(datapath, "Annotation/{}/{}_{}.xml".format(label, label, fname)))
    xml_data = tree.getroot()
    xmlstr = ElementTree.tostring(xml_data, encoding="utf-8", method="xml")
    annotation = dict(xmltodict.parse(xmlstr))['annotation']

    width = int(annotation["size"]["width"])
    height = int(annotation["size"]["height"])

    # iterate objects
    objects = annotation["object"]

    if type(objects) != list:
        mask = np.zeros((height, width), dtype=int)

        mask[int(objects['bndbox']['ymin']):int(objects['bndbox']['ymax']), int(objects['bndbox']['xmin']):int(objects['bndbox']['xmax'])] = 1

        binary_mask[objects['name']] = mask

    else:
        for object in annotation['object']:
            if type(object) == collections.OrderedDict:
                if object['name'] in binary_mask.keys():
                    mask = binary_mask[object['name']]
                else:
                    mask = np.zeros((height, width), dtype=np.uint8)

                mask[int(object['bndbox']['ymin']):int(object['bndbox']['ymax']),
                     int(object['bndbox']['xmin']):int(object['bndbox']['xmax'])] = 1

                binary_mask[object['name']] = mask

    # preprocess binary masks to fit shape of image data
    for key in binary_mask.keys():
        binary_mask[key] = cv2.resize(binary_mask[key],
                                      (224, 224),
                                      interpolation=cv2.INTER_NEAREST).astype(np.int)[:, :, np.newaxis]

    return binary_mask


class custom_transform(torch.nn.Module):
    """ Custom transformation to handle image processing and binary mask processing simultaneously. """
    
    image_transform = transforms.Compose([transforms.Resize((224, 224)),
                                          transforms.ToTensor(),
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    
    def __init__(self, datapath):
        
        super().__init__()
        self.datapath = datapath
        
    def __call__(self, img):
        return {"data": self.image_transform(img["data"]), "mask": load_binary_mask(img["path"], self.datapath)}
    
    
def custom_collate(batch):
    """ Collate function to pack the image, mask and label data accordingly. """
    x = []
    y = []
    mask = []
    
    for dp in batch:
        x.append(dp[0]["data"])
        mask.append(dp[0]["mask"])
        y.append(dp[1])
    
    return (torch.stack(x, 0), mask), torch.tensor(y) 

datapath = "../../data/imagenet/imagenet/"

mask_set = torchvision.datasets.DatasetFolder(os.path.join(datapath, "bboxes_images/"), 
                                   loader=load_sample, 
                                   extensions=(".jpeg", "png"),
                                   transform=custom_transform(datapath))

test_loader = torch.utils.data.DataLoader(mask_set, shuffle=True, batch_size=24, collate_fn=custom_collate)

# Load a batch of inputs and outputs to use for evaluation.
(x_batch, d_batch), y_batch = iter(test_loader).next()
print(x_batch.shape)
x_batch, y_batch = x_batch.to(device), y_batch.to(device)

torch.Size([24, 3, 224, 224])


In [4]:
# Save x_batch and y_batch as numpy arrays that will be used to call metric instances.
x_batch, y_batch = x_batch.cpu().numpy(), y_batch.cpu().numpy()
print(x_batch.shape)

(24, 3, 224, 224)


In [12]:
s_batch = []

for mask_dict in d_batch:
    s_batch.append(mask_dict[list(mask_dict.keys())[0]][:, :, 0])

s_batch = np.array(s_batch)
print(s_batch.shape)

(24, 224, 224)


In [13]:
# Return effective complexity scores in an one-liner - by calling the metric instance.
quantus.PointingGame(**{
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   s_batch=s_batch,
   **{"explain_func": quantus.explain, "method": "Saliency", "device": device})

[1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1]

In [14]:
# Return attribution localization scores in an one-liner - by calling the metric instance.
quantus.AttributionLocalization(**{
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   s_batch=s_batch,
   **{"explain_func": quantus.explain, "method": "Saliency", "device": device})

[0.40025213,
 0.20398511,
 0.17013627,
 0.997503,
 0.833114,
 0.34701458,
 0.140623,
 0.32279322,
 0.42278847,
 0.28827336,
 0.4164221,
 0.93985087,
 0.6603087,
 0.7212441,
 0.2014988,
 0.5006696,
 0.22828004,
 0.97880894,
 0.020070205,
 0.80277777,
 0.7964626,
 0.39282787,
 0.9424784,
 0.40008068]

In [15]:
# Return tki scores in an one-liner - by calling the metric instance.
quantus.TopKIntersection(**{
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   s_batch=s_batch,
   **{"explain_func": quantus.explain, "method": "Saliency", "device": device})

[0.9530000000000001,
 0.063,
 0.07100000000000001,
 0.999,
 0.883,
 0.385,
 0.08,
 0.322,
 0.648,
 0.764,
 0.468,
 1.0,
 0.987,
 0.997,
 0.135,
 0.9520000000000001,
 0.10200000000000001,
 0.98,
 0.035,
 0.9520000000000001,
 0.974,
 0.854,
 0.998,
 0.894]

In [16]:
# Return relevane rank accuracy scores in an one-liner - by calling the metric instance.
quantus.RelevanceRankAccuracy(**{
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   s_batch=s_batch,
   **{"explain_func": quantus.explain, "method": "Saliency", "device": device})

[0.5405813953488372,
 0.19370182056749222,
 0.1550314465408805,
 0.9957359064702114,
 0.8189867797947908,
 0.36818181818181817,
 0.12523744911804613,
 0.3322395933860902,
 0.45998082913970767,
 0.5196112460951059,
 0.4452092511013216,
 0.9074933526710176,
 0.6364992796871785,
 0.7608746546799644,
 0.1761954261954262,
 0.5968265416516408,
 0.2061711079943899,
 0.9777804957599479,
 0.03368421052631579,
 0.770242711474939,
 0.7538870873867133,
 0.5091525423728813,
 0.9091010235870048,
 0.6342383107088989]

In [199]:
# Return relevane mass accuracy scores in an one-liner - by calling the metric instance.
quantus.RelevanceMassAccuracy(**{
})(model=model, 
   x_batch=x_batch,
   y_batch=y_batch,
   a_batch=None,
   s_batch=s_batch,
   **{"explain_func": quantus.explain, "method": "Saliency", "device": device})

AttributeError: module 'quantus' has no attribute 'RelevanceMassAccuracy'